# FE approximation of the Navier-Stokes problem with Newton linearization

We consider again the steady Navier-Stokes problem defined in the domain $\Omega := {[0,1]}^2$ by the following system of PDEs and Dirichlet BCs: 
$$
\begin{cases}
  ({\boldsymbol{u}}\cdot\boldsymbol{\nabla})\ \boldsymbol{u}
  -\dfrac{1}{\text{Re}}\boldsymbol{\Delta} \boldsymbol{u} + \boldsymbol{\nabla} p = 0, 
  \qquad&\text{in }\Omega, \\
  \boldsymbol{\nabla} \cdot \boldsymbol{u} = 0, 
  \qquad &\text{in }\Omega, \\
  \boldsymbol{u} = 1\boldsymbol{i} + 0\boldsymbol{j}, \qquad&\text{on }\Gamma^{\text{up}} = \{ 0 \le x \le 1, \, y=1 \}, \\
  \boldsymbol{u} = \boldsymbol{0}, \qquad&\text{on }\partial\Omega \backslash \Gamma^{\text{up}}.
\end{cases}
$$

The linearized problem for the Netwon's method is given by
$$
  a(\boldsymbol{u}_k,\boldsymbol{v}) 
  + c(\boldsymbol{u}_k,\boldsymbol{u}_{k-1},\boldsymbol{v})
  + c(\boldsymbol{u}_{k-1},\boldsymbol{u}_k,\boldsymbol{v}) 
  - b(\boldsymbol{v},p_k) + b(\boldsymbol{u}_k,q)
  =
c(\boldsymbol{u}_{k-1},\boldsymbol{u}_{k-1},\boldsymbol{v}),
$$
where
$$
  a(\boldsymbol{u},\boldsymbol{v}):= \frac{1}{\text{Re}}
  \int_\Omega\boldsymbol{\nabla}{\boldsymbol{u}}:\boldsymbol{\nabla}{\boldsymbol{v}}\, d\boldsymbol{x},\qquad
  b(\boldsymbol{u},p) := -\int_\Omega \boldsymbol{\nabla}\cdot{\boldsymbol{u}}\ p \,d\boldsymbol{x},\qquad
c(\boldsymbol{u},\boldsymbol{v},\boldsymbol{w}) := \int_\Omega (\boldsymbol{u}\cdot\boldsymbol{\nabla})\boldsymbol{v}\ \boldsymbol{w} \, d\vec{x}.
$$

## Numerical approximation with FEniCS

In [ ]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

from fenics import *

Using the stable pair of spaces $\mathbb{P}^2/\mathbb{P}^1$ solve the problem using Netwon's method. Consider a stopping criterion based on the relative increment
$$
\frac{||\boldsymbol{u}_k-\boldsymbol{u}_{k-1}||_{H^1}}{||\boldsymbol{u}_{k-1}||_{H^1}}
+\frac{||\boldsymbol{p}_k-\boldsymbol{p}_{k-1}||_{L^2}}{||\boldsymbol{p}_{k-1}||_{L^2}}
\le 10^{-6}.
$$

*   Try to solve the problem with different values of Reynolds number ($\text{Re}=10,100,1000$). 
*   Assess the convergence performance of the iterative method.

In [ ]:
# 1. mesh generation
n = 15
mesh = UnitSquareMesh(n, n, 'crossed')

# 2. finite element space
V = VectorElement('CG', mesh.ufl_cell(), 2)
Q = FiniteElement('CG', mesh.ufl_cell(), 1)
X = FunctionSpace(mesh, V*Q)

u_boundary = Expression((
        'near(x[1], 1) ? 1.0 : 0.0',
        '0'
    ), degree=0)

def boundary(x, on_boundary):
  return on_boundary

def origin(x, on_boundary):
  return near(x[0], 0) and near(x[1], 0)

bc = [DirichletBC(X.sub(0), u_boundary, boundary),
      DirichletBC(X.sub(1), Constant(0), origin, 'pointwise')]

# 3. problem definition
def solve_linear(X, advection, Re, f, bc):
  u, p = TrialFunctions(X)
  v, q = TestFunctions(X)

  a = (inner(grad(u), grad(v)) / Re - p * div(v) + div(u) * q) * dx
  L = dot(f, v) * dx
  if advection:
    # only the first term was present in the fixed point iterative method
    a += (dot(grad(u) * advection, v) + dot(grad(advection) * u, v)) * dx
    L += dot(grad(advection) * advection, v) * dx
  
  x = Function(X)
  solve(a == L, x, bc)
  
  return x.split()

# 4. solution
Re = Constant(100)
f = Constant((0, 0))

# compute initial guess
uh, ph = solve_linear(X, None, Re, f, bc)

niter = 50
tolerance = 1e-6
for i in range(niter):
  uh_old, ph_old = uh, ph
  uh, ph = solve_linear(X, uh_old, Re, f, bc)
  
  error = (errornorm(uh, uh_old, 'H1') / norm(uh_old, 'H1') +
           errornorm(ph, ph_old, 'L2') / norm(ph_old, 'L2'))
  print('step {}: {:.3e}'.format(i, error))
  if error < tolerance:
    break

uh.rename('velocity', 'velocity')
ph.rename('pressure', 'pressure')

File('velocity.pvd') << uh
File('pressure.pvd') << ph

step 0: 9.746e-01
step 1: 1.199e-01
step 2: 1.329e-03
step 3: 2.000e-07


### Streamline diffusion stabilizition of the $\mathbb{P}^1-\mathbb{P}^1$ FEM

If the pair of spaces $\mathbb{P}^1/\mathbb{P}^1$ is considered, then the linearized problem with the stabilized terms is given by
\begin{multline*}
  a(\boldsymbol{u}_k,\boldsymbol{v}) 
  + c(\boldsymbol{u}_k,\boldsymbol{u}_{k-1},\boldsymbol{v})
  + c(\boldsymbol{u}_{k-1},\boldsymbol{u}_k,\boldsymbol{v}) 
  + b(\boldsymbol{v},p_k) - b(\boldsymbol{u}_k,q) + \\
  \sum_{K\in\mathcal{T}_h} \delta_K \left[{\left(
    (\boldsymbol{u}_k\cdot\boldsymbol{\nabla})
    \boldsymbol{u}_{k-1}
    +(\boldsymbol{u}_{k-1}\cdot\boldsymbol{\nabla})
    \boldsymbol{u}_{k}
    + \boldsymbol{\nabla}{p},
    (\boldsymbol{v}\cdot\boldsymbol{\nabla})
    \boldsymbol{u}_{k-1}
    +(\boldsymbol{u}_{k-1}\cdot\boldsymbol{\nabla})
    \boldsymbol{v}
    + \boldsymbol{\nabla}{q}\right)}_K
    + (\boldsymbol{\nabla}\cdot{\boldsymbol{u}_k}, 
    \boldsymbol{\nabla}\cdot{\boldsymbol{v}})_K\right]= \\ 
c(\boldsymbol{u}_{k-1},\boldsymbol{u}_{k-1},\boldsymbol{v})
+ \sum_{K\in\mathcal{T}_h} \delta_K {\left(
  (\boldsymbol{u}_{k-1}\cdot\boldsymbol{\nabla})
  \boldsymbol{u}_{k-1},
  (\boldsymbol{v}\cdot\boldsymbol{\nabla})
  \boldsymbol{u}_{k-1}
  +(\boldsymbol{u}_{k-1}\cdot\boldsymbol{\nabla})
  \boldsymbol{v}
  + \boldsymbol{\nabla}{q}\right)}_K,
\end{multline*}
where ${(\cdot, \cdot)}_K$ denotes the $L^2$ inner product over the element $K$. The stabilization parameters are defined by
$$
\delta_K = 
\delta \min{\left\{ \frac{h_K}{2 ||\boldsymbol{u}||}_K, \frac{h^2_K \text{Re}}{24} \right\}},\qquad \delta > 0.
$$

In [ ]:
# 1. mesh generation
n = 15
mesh = UnitSquareMesh(n, n, 'crossed')

# 2. finite element space
V = VectorElement('CG', mesh.ufl_cell(), 1)
Q = FiniteElement('CG', mesh.ufl_cell(), 1)
X = FunctionSpace(mesh, V*Q)

u_boundary = Expression((
        'near(x[1], 1) ? 1.0 : 0.0',
        '0'
    ), degree=0)

def boundary(x, on_boundary):
  return on_boundary

def origin(x, on_boundary):
  return near(x[0], 0) and near(x[1], 0)

bc = [DirichletBC(X.sub(0), u_boundary, boundary),
      DirichletBC(X.sub(1), Constant(0), origin, 'pointwise')]

# 3. problem definition
def solve_linear(X, advection, Re, f, bc):
  u, p = TrialFunctions(X)
  v, q = TestFunctions(X)

  a = (inner(grad(u), grad(v)) / Re - p * div(v) + div(u) * q) * dx
  L = dot(f, v) * dx
  if advection:
    a += (dot(grad(u) * advection, v) + dot(grad(advection) * u, v)) * dx
    L += dot(grad(advection) * advection, v) * dx
    
    A = lambda u, p: (grad(u) * advection) + (grad(advection) * u) + grad(p)
    A_SS = lambda u, p: (grad(u) * advection) + (grad(advection) * u) + grad(p)
  
    h = CellDiameter(X.mesh())
    anorm = sqrt(dot(advection, advection))
    tau_K = 0.5 * h / conditional(anorm * h * Re > 12, anorm, 12 / h / Re)
  
    a += tau_K * (dot(A(u, p), A_SS(v, q)) + div(u) * div(v)) * dx
    L += tau_K * (dot(f + grad(advection) * advection, A_SS(v, q))) * dx
  
  x = Function(X)
  solve(a == L, x, bc)
  
  return x.split()

# 4. solution
Re = Constant(100)
f = Constant((0, 0))

uh, ph = solve_linear(X, None, Re, f, bc)

niter = 50
tolerance = 1e-6
for i in range(niter):
  uh_old, ph_old = uh, ph
  uh, ph = solve_linear(X, uh_old, Re, f, bc)
  
  error = (errornorm(uh, uh_old, 'H1') / norm(uh_old, 'H1') +
           errornorm(ph, ph_old, 'L2') / norm(ph_old, 'L2'))
  print('step {}: {:.3e}'.format(i, error))
  if error < tolerance:
    break

uh.rename('velocity_stab', 'velocity')
ph.rename('pressure_stab', 'pressure')

File('velocity_stab.pvd') << uh
File('pressure_stab.pvd') << ph

step 0: 1.266e+00
step 1: 1.047e-01
step 2: 9.925e-04
step 3: 9.449e-06
step 4: 1.828e-07
